# Pluralism in Europe - Part 1: Human Values


## Abstract
Taking a broad definition of pluralism, pluralists are people believing that diversity is a good thing for society. In this first data analysis, I've tried to understand who these people are and what values they are sharing. 

To this end, I've used freely available results from the European Social Survey (2018 edition) and in particular its 21-item measure of human values designed by Shalom H. Schwartz.

I’ve highlighted cultural similarities across 27 European countries and have raised some important questions regarding diversity and its multiple meanings.

- ESS Round 9: European Social Survey Round 9 Data (2018). Data file edition 3.1. NSD - Norwegian Centre for Research Data, Norway – Data Archive and distributor of ESS data for ESS ERIC. http://dx.doi.org/10.21338/NSD-ESS9-2018
- The Human Value Scale: https://www.europeansocialsurvey.org/docs/findings/ESS_Findings_HVS.pdf

**License**

The content of this project is licensed under the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License (CC BY-NC-SA 4.0), and the underlying source code is licensed under GPL-3.0 License.




## What is diversity?
Diversity can take many forms: diversity of ideas, opinions and lifestyles, ethnic, gender and sexual diversity to name just a few. 

Pluralists are people who value not just specific differences but all of them. It is thus not enough to sympathize with a specific community or minority group to be considered a pluralist. Pluralists value difference whatever form it takes.

Like most surveys, The European Social Survey probes people’s attitudes towards specific sociocultural groups (e.g. gays and lesbians, immigrants, ethnic minorities) but does not study pluralism per se. 

Thankfully, some of its 21 statements designed to study human values could give us an indication. In this section of the survey, respondents are asked to rate from 1 (very much like me) to 6 (not like me at all) how much they identify with some generic statements. Here are some examples:
1. Thinking up new ideas and being creative is important to her/him. She/he likes to do things in her/his own original way.
2. She/he likes surprises and is always looking for new things to do. She/he thinks it is important to do lots of different things in life.
3. It is important to her/him to listen to people who are different from her/him. Even when she/he disagrees with them, she/he still wants to understand them.
4. It is important to her/him to make her/his own decisions about what she/he does. She/he likes to be free and not depend on others.

Comparing these statements with our definition of pluralism, statement #3 (‘ipudrst’ variable) is particularly promising: not only does it express a general interest in difference, but it does explicitly value disagreement, which implies more than just sympathy towards specific sociocultural groups.


## Initialisation & data cleansing

In [1]:
# Data source: European Social Survey Cumulative File, ESS 1-9 (2020).
# Data file edition 1.1.
# NSD - Norwegian Centre for Research Data, Norway - Data Archive and distributor of ESS data for ESS ERIC.
# www.europeansocialsurvey.org

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
# import raw dataset
ess_rawdataset = pd.read_csv("data/ESS1-9e01_1.csv", low_memory=False)

In [4]:
# extract relevant features 
# descriptions are simplified, please refer to ESS documentation for full descriptions
ess_dataset = ess_rawdataset[['cntry', # country
                              'anweight', # analysis weight that makes all necessary survey corrections                            

                              # human values from 1 (very much like me) to 6 (not like me at all)                              
                              'ipcrtiv','imprich','ipeqopt','ipshabt',
                              'impsafe','impdiff','ipfrule','ipudrst',
                              'ipmodst','ipgdtim','impfree','iphlppl',
                              'ipsuces','ipstrgv','ipadvnt','ipbhprp',
                              'iprspot','iplylfr','impenv','imptrad',
                              'impfun'
                              # line1: important to be creative, rich, treat equally, be admired, 
                              # 2: to live safe, try different things, follow rules, understand different people
                              # 3: to be modest, have good time, be free, help people
                              # 4: to be successful, that government is strong, to seek adventures, behave properly
                              # 5: to get respect, to be loyal to friends, to care for environment, to follow traditions
                              # 6: to seek fun
                              ]]



In [5]:
print('Total number of respondents: '+str(ess_dataset.shape[0]))

Total number of respondents: 46276


In [6]:
# for the sake of this exploration, remove respondents who didn't provide an answer to any relevant question
ess_dataset = ess_dataset.dropna()                                
ess_dataset.drop(ess_dataset[(ess_dataset.ipcrtiv > 6)| 
                              (ess_dataset.imprich > 6)| 
                              (ess_dataset.ipeqopt > 6)| 
                              (ess_dataset.ipshabt > 6)| 
                              (ess_dataset.impsafe > 6)| 
                              (ess_dataset.impdiff > 6)| 
                              (ess_dataset.ipfrule > 6)| 
                              (ess_dataset.ipudrst > 6)| 
                              (ess_dataset.ipmodst > 6)| 
                              (ess_dataset.ipgdtim > 6)| 
                              (ess_dataset.impfree > 6)| 
                              (ess_dataset.iphlppl > 6)| 
                              (ess_dataset.ipsuces > 6)| 
                              (ess_dataset.ipstrgv > 6)| 
                              (ess_dataset.ipadvnt > 6)| 
                              (ess_dataset.ipbhprp > 6)| 
                              (ess_dataset.iprspot > 6)| 
                              (ess_dataset.iplylfr > 6)| 
                              (ess_dataset.impenv > 6)| 
                              (ess_dataset.imptrad > 6)| 
                              (ess_dataset.impfun > 6)].index, inplace=True)
print('Total number of respondents after cleansing: '+str(ess_dataset.shape[0]))

Total number of respondents after cleansing: 42425


(only a fairly small number of data entries were lost in the cleansing) 

## Comparing pluralism and other human values 

In this main section, I extract what are the most correlated statements to the one I have picked as a good indicator of pluralism (ipudrst: “It is important to her/him to listen to people who are different from her/him”).

I perform the analysis first for Europe as a whole and then repeat the same procedure per country.

In [7]:
# Select features that are relevant for our analysis
features_human_values = [                             
                              'cntry','ipcrtiv','imprich','ipeqopt','ipshabt',
                              'impsafe','impdiff','ipfrule','ipudrst',
                              'ipmodst','ipgdtim','impfree','iphlppl',
                              'ipsuces','ipstrgv','ipadvnt','ipbhprp',
                              'iprspot','iplylfr','impenv','imptrad',
                              'impfun'
                              ]

# Create a dataframe to compare human values by country
countries_series = pd.DataFrame()  

# Create a global dataframe including all European respondents and explore correlation
ess_dataset_global_human_values = ess_dataset[features_human_values].copy()
del ess_dataset_global_human_values['cntry']
corr_matrix = ess_dataset_global_human_values.corr().abs()
corr_matrix.sort_values(by=['ipudrst'],axis=0,ascending=False, inplace=True)
corr_matrix=corr_matrix[:4].T.loc['ipudrst'].reset_index(level=0)
corr_matrix.rename(columns=({'index': 'Europe'}),inplace=True)
corr_matrix['Europe']=corr_matrix['Europe']+": "+round(corr_matrix['ipudrst'],2).astype(str)
countries_series=countries_series.append(corr_matrix['Europe'])

# Repeat the same procedure per country 
for country in ess_dataset["cntry"].unique():
    ess_dataset_country = ess_dataset[features_human_values].copy().loc[ess_dataset["cntry"] == country]
    del ess_dataset_country['cntry']
    corr_matrix = ess_dataset_country.corr().abs()
    corr_matrix.sort_values(by=['ipudrst'],axis=0,ascending=False, inplace=True)
    # add median 
    corr_matrix=corr_matrix[:4].T.loc['ipudrst'].reset_index(level=0)
    corr_matrix.rename(columns=({'index': country}),inplace=True)
    corr_matrix[country]=corr_matrix[country]+": "+round(corr_matrix['ipudrst'],2).astype(str)
    countries_series=countries_series.append(corr_matrix[country])


countries_series=countries_series.drop(columns=[0])

The table below lists top 3 most correlated human values to 'ipudrst' per country, along with their correlation coefficient. I have added arbitrary colours to better visualise most frequent human values.

In [8]:
# Display results 
countries_series.sort_values(by=1,ascending=False,inplace=True)
def colour(s):
    return ['background-color: #f3ff99' if 'iphlppl' in v
            else 'background-color: #ffcc99' if 'iplylfr' in v
            else 'background-color: #ff9999' if 'ipeqopt' in v
            else 'background-color: #bd99ff' if 'impenv' in v
            else 'background-color: #99ccff' if 'ipmodst' in v
            else '' for v in s]
countries_series.style.apply(colour)

,1,2,3
LV,iplylfr: 0.4,iphlppl: 0.33,ipeqopt: 0.3
EE,iplylfr: 0.29,iphlppl: 0.28,ipeqopt: 0.23
LT,iplylfr: 0.29,ipeqopt: 0.28,iphlppl: 0.28
DE,iplylfr: 0.27,ipeqopt: 0.25,iphlppl: 0.24
CZ,iphlppl: 0.47,ipmodst: 0.43,ipfrule: 0.37
IT,iphlppl: 0.45,impenv: 0.44,ipmodst: 0.44
HR,iphlppl: 0.44,ipeqopt: 0.4,iplylfr: 0.4
PL,iphlppl: 0.43,iplylfr: 0.38,impenv: 0.38
IE,iphlppl: 0.43,ipeqopt: 0.4,iplylfr: 0.38
Europe,iphlppl: 0.42,iplylfr: 0.38,ipeqopt: 0.37


## Interpretation
**Results are consistent across Europe, with 3 statements dominating the table:**
1. Iphlppl (25 entries): It's very important to her/him to help the people around her/him. She/he wants to care for their well-being.
2. Ipeqopt (21 entries): She/he thinks it is important that every person in the world should be treated equally. She/he believes everyone should have equal opportunities in life.
3. Iplylfr (15 entries): It is important to her/him to be loyal to her/his friends. She/he wants to devote herself/himself to people close to her/him.
4. Ipmodst (7 entries): It is important to her/him to be humble and modest. She/he tries not to draw attention to herself/himself.
5. Impenv (6 entries): She/he strongly believes that people should care for nature. Looking after the environment is important to her/him.

While there seems to be cultural differences across Europe (in particular between countries of protestant tradition and the rest of Europe), a full description and explanation of these differences is out of this exploration’s scope (contributions are welcomed!).

I should point out that I have not used ESS analysis weights and human value correction factors in this analysis, which are both necessary in order to make accurate country comparisons. This might be crucial given that differences between top correlation coefficients can be very small. 
https://www.europeansocialsurvey.org/docs/methodology/ESS_weighting_data_1_1.pdf
https://www.europeansocialsurvey.org/docs/methodology/ESS_computing_human_values_scale.pdf


![Most correlated values to pluralism](Most_correlated_value_to_pluralism.png)

**Results are consistent with the human value model used in the survey.** 

The Schwartz (1992) Value Survey (SVS) is “currently the most widely used by social and cross-cultural psychologists for studying individual differences in values”. It is based on 10 human values that are derived from 21 statements included in the European Social Survey. 

The statement I have used for my analysis (ipudrst: “It is important to her/him to listen to people who are different from her/him”) is part of the model’s Universalism human value: “Understanding, appreciation, tolerance and protection for the welfare of all people and for nature”.

- 'Ipeqopt' and 'Impenv' are also part of the model’s Universalism human value, which explains their high correlation. 
- 'Iphlppl' and 'Iplylfr' are both part of the model’s Benevolence human value, which it predicts is the closest to Universalism (along with Self-direction).
- 'Ipmodst' belongs to the model’s Tradition human value, which is a bit further away, just after Benevolence.

https://www.europeansocialsurvey.org/docs/methodology/core_ess_questionnaire/ESS_core_questionnaire_human_values.pdf

**Results hint at tension between what I call principled and sociable pluralists.**

On the one hand, helping people around us and being loyal to our friends requires empathy and the ability to understand people different from us. On the other hand, friends and people around us are probably more similar to us than the average population. This is what I call sociable pluralism.

Similarly, believing in environment protection and equal opportunities in life implies that we are all in the same boat, whatever our differences.  But it is a more abstract and principled understanding of pluralism than its sociable version. 

Both principled and familiar pluralism are positive values, but they each have ethical limitations.  Based on our results, they seem to often go hand in hand but not with the same importance, it might indicate some fundamental cultural differences on how respondents articulate and appropriate the notion of pluralism. 

**In the next part of my exploration, I will be looking more closely at the relation between pluralism and attitude towards gender and sexual diversity.**
[Pluralism in Europe - Part 2: Sex and Gender](2_Pluralism_Sex&Gender.ipynb)